In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import yaml

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [3]:
import torch
from torch.utils.data import DataLoader
from src import LazyWindowedDataset

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=False, num_workers=1, drop_last=True)

In [4]:
from models import BaselineAutoencoder
from src import evaluate, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, _, _, _, _, _, _ = load_model_checkpoint("checkpoints/BrushlessMotorBaseline.pt", BaselineAutoencoder, optimizer_class=None)
model.to(device)
criterion = torch.nn.MSELoss()

In [5]:
from torchinfo import summary
summary(model, (batch_size, 5620))

Layer (type:depth-idx)                   Output Shape              Param #
BaselineAutoencoder                      [128, 5620]               --
├─Sequential: 1-1                        [128, 16]                 --
│    └─Linear: 2-1                       [128, 2048]               11,511,808
│    └─ReLU: 2-2                         [128, 2048]               --
│    └─Linear: 2-3                       [128, 2048]               4,196,352
│    └─ReLU: 2-4                         [128, 2048]               --
│    └─Linear: 2-5                       [128, 2048]               4,196,352
│    └─ReLU: 2-6                         [128, 2048]               --
│    └─Linear: 2-7                       [128, 16]                 32,784
├─Sequential: 1-2                        [128, 5620]               --
│    └─Linear: 2-8                       [128, 2048]               34,816
│    └─ReLU: 2-9                         [128, 2048]               --
│    └─Linear: 2-10                      [128, 2048]   

![alt text](figures/brushlessMotorHighlightedTable.png)

In [5]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Overall S+T | Loss: {loss:.4f}, AUC: {auc:.4f}")

Overall S+T | Loss: 0.8567, AUC: 0.5883


In [6]:
acc_loss, acc_auc = evaluate(model, test_loader, criterion, sensors_to_test=['acc'])
print(f"Acc s+T | Loss: {acc_loss:.4f}, AUC: {acc_auc:.4f}")

Acc s+T | Loss: 0.4076, AUC: 0.6800


In [7]:
gyro_loss, gyro_auc = evaluate(model, test_loader, criterion, sensors_to_test=['gyro'])
print(f"Gyro S+T | Loss: {gyro_loss:.4f}, AUC: {gyro_auc:.4f}")

Gyro S+T | Loss: 0.2563, AUC: 0.5709


In [8]:
mic_loss, mic_auc = evaluate(model, test_loader, criterion, sensors_to_test=['mic'])
print(f"Mic S+T | Loss: {mic_loss:.4f}, AUC: {mic_auc:.4f}")

Mic S+T | Loss: 0.3634, AUC: 0.4801
